In [1]:
import os
import pandas as pd
import pandas as pd
import numpy as np
import cvxpy as cp
import yfinance as yf
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import gurobipy as gp
from gurobipy import GRB
from scipy.stats import norm

In [2]:
# Load the stock data.
file_path = '..\data\s&p500\s&p500_2016to2022_stock_data.csv'
stock_data = pd.read_csv(file_path, index_col='date', parse_dates=True)

In [3]:
stock_data.head()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,2012.660034,40.689999,40.910000,152.240005,26.337500,57.610001,85.239998,NaN,42.930000,22.950001,...,35.700001,77.459999,58.860001,27.140976,36.080002,51.912292,98.844658,66.489998,26.709999,47.270000
2016-01-05,2016.709961,40.549999,40.520000,151.199997,25.677500,57.369999,85.000000,NaN,42.919998,23.033333,...,36.060001,78.120003,60.060001,27.088274,36.070000,51.782890,100.902916,64.820000,26.420000,48.009998
2016-01-06,1990.260010,40.730000,41.230000,147.199997,25.174999,57.380001,85.300003,NaN,42.560001,23.070000,...,36.439999,77.470001,59.189999,26.745718,35.619999,51.416248,101.339806,62.230000,25.650000,48.020000
2016-01-07,1943.089966,39.000000,40.450001,148.830002,24.112499,57.209999,81.919998,NaN,41.540001,23.046667,...,36.580002,76.230003,58.669998,26.007904,34.700001,49.662113,99.009712,59.410000,24.879999,46.560001
2016-01-08,1922.030029,38.590000,40.369999,145.559998,24.240000,55.650002,84.580002,NaN,40.669998,22.806667,...,36.180000,74.690002,56.990002,25.270092,34.369999,48.986340,98.592232,59.250000,24.600000,45.880001


In [4]:
# Calculate daily returns
stock_returns = stock_data.pct_change().dropna(how='all')
stock_returns.tail()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-12-19,-0.009008,-0.015740,-0.025761,0.005280,-0.015910,0.006294,0.000787,0.0,-0.002993,-0.008029,...,-0.007437,0.004489,0.002621,-0.071099,-0.013011,-0.007652,-0.004937,-0.020930,-0.007163,-0.008791
2022-12-20,0.001037,0.006669,0.004006,-0.010788,-0.000529,-0.005635,-0.001337,0.0,-0.000938,0.028081,...,0.001297,0.014453,0.009150,0.042523,-0.001751,0.001636,0.003761,0.014115,0.008526,-0.005028
2022-12-21,0.014868,0.014602,0.039904,0.020878,0.023809,0.010151,0.000604,0.0,0.015494,0.017995,...,0.013239,0.012841,0.014573,0.009517,0.026411,0.001478,0.013155,0.014930,0.031216,0.017197
2022-12-22,-0.014452,-0.007196,-0.036071,-0.001476,-0.023773,0.006535,0.000000,0.0,-0.001017,-0.008049,...,-0.005397,-0.020174,0.004788,-0.012795,-0.012056,0.000311,-0.002282,-0.013275,0.014925,0.000759
2022-12-23,0.005868,0.001476,0.011943,0.008446,-0.002798,-0.001041,0.000000,0.0,0.001389,0.008433,...,0.012852,0.026445,0.011118,-0.003411,-0.000728,0.000621,-0.000789,0.002869,0.003521,0.005033


In [5]:
stock_returns['^GSPC'].isna().sum()

0


# Experiment setup
- daily return을 기준으로 index tracking 설정
- 월별 리벨런싱 가정
- 2006/01/01 ~ 2022-12/30
- k act: 10% 20% 30% 40% 50% - 전체 주식의 % 

# Algorithm

### Gurobi with adaptive loss 예제

In [6]:
import gurobipy as gp
from gurobipy import GRB

# 데이터 초기화
max_iterations = 10  # 최대 반복 횟수
epsilon = 1e-6  # 수렴 조건
w = 1.0  # 초기 가중치
previous_obj_value = float('inf')  # 초기 Objective Function 값

# 모델 생성
model = gp.Model("adaptive_weights")

# 변수 추가
x = model.addVar(name="x")
y = model.addVar(name="y")

# 제약 조건 추가
model.addConstr(x + 2 * y <= 4, "c0")
model.addConstr(2 * x + y <= 5, "c1")

# 반복 프로세스
for iteration in range(max_iterations):
    # 목표 함수 설정
    model.setObjective(w * (x + y), GRB.MAXIMIZE)
    
    # 최적화 수행
    model.optimize()
    
    # 결과 출력
    if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
        for v in model.getVars():
            print(f'Iteration {iteration}, {v.varName}: {v.x}')
        print(f'Iteration {iteration}, Obj: {model.objVal}')
        
        # 가중치 업데이트 (예: 목표 함수 값의 절대값을 가중치로 사용)
        new_w = abs(model.objVal)
        
        # 수렴 확인
        if abs(previous_obj_value - model.objVal) < epsilon:
            print(f"Converged at iteration {iteration}")
            break
        
        # 업데이트된 가중치 및 이전 목표 함수 값 갱신
        w = new_w
        previous_obj_value = model.objVal
    else:
        print(f"Optimization was stopped at iteration {iteration}")
        break

# 최종 결과 출력
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    for v in model.getVars():
        print(f'Final {v.varName}: {v.x}')
    print(f'Final Obj: {model.objVal}')
else:
    print("최적 해를 찾지 못했습니다.")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-25
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xb1b4833e
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+00]
Presolve time: 0.02s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+30   3.000000e+30   2.000000e+00      0s
       2    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.04 seconds (0.00 work units)
Optimal objective  3.000000000e+00
Iteration 0, x: 2.0
Iteration 0, y: 1.0
Iteration 0, Obj: 3.0
Gurobi 

## CARD

In [7]:
def card(X, y, kact, time_limit=1200):
    p = X.shape[1]
    
    # Define variables
    beta = cp.Variable(p)
    z = cp.Variable(p, boolean=True)  # Binary variables for cardinality constraint
    
    # Objective function
    objective = cp.Minimize(cp.norm2(y - X @ beta) ** 2)
    
    # Constraints
    constraints = [
        cp.sum(beta) == 1,  # Sum of weights is 1
        beta >= 0,  # All weights beta are non-negative
        cp.sum(z) <= kact,  # Number of non-zero weights is less than or equal to kact
        beta <= z  # Link binary variables with beta
    ]
    
    # Problem definition
    prob = cp.Problem(objective, constraints)
    
    # Solve the problem
    prob.solve(solver=cp.GUROBI, TimeLimit=time_limit)
    
    beta_opt = beta.value
    
    return beta_opt

In [8]:
stock_stocks = stock_returns.drop(columns=['^GSPC'])
stock_index = stock_returns['^GSPC']

train_data = stock_stocks.iloc[:250]

# Filter out columns with NaN values in the current window
valid_assets_train = train_data.dropna(axis=1)
valid_assets = valid_assets_train.columns.intersection(valid_assets_train.columns)

X_train = valid_assets_train[valid_assets].values
y_train = stock_index.iloc[:250].values

In [9]:
# # CARD
beta_card = card(X_train, y_train, kact=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)


c:\Users\zin\Developer\RankingMeetsIndexTracking\.venv\lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [ ]:
non_negative_count

## ALASSO Algorithm
- make initial beta using OLS
- can not find propal lambda in the context
- FIX!: ALASSO solution is always converge to 1,0,0,0 -> shrinking is too strong

In [10]:
from sklearn.linear_model import Lasso

def alasso(X, y, kact, time_limit=1200, lambdas=1):
    p = X.shape[1]
    
    # Initial Lasso to get initial weights
    lasso = Lasso(alpha=0.1, max_iter=10000)
    lasso.fit(X, y)
    beta_init = lasso.coef_
    
    # Adaptive weights
    weights = 1 / (np.abs(beta_init) + 1e-6)
    
    # Define variables
    beta = cp.Variable(p)
    z = cp.Variable(p, boolean=True)  # Binary variables for cardinality constraint
    
    # Objective function
    objective = cp.Minimize(cp.norm2(y - X @ beta) ** 2 + lambdas * cp.sum(weights @ cp.abs(beta)))
    
    # Constraints
    constraints = [
        cp.sum(beta) == 1,  # Sum of weights is 1
        beta >= 0,  # All weights beta are non-negative
        cp.sum(z) == kact,  # Number of non-zero weights is less than or equal to kact
        beta <= z  # Link binary variables with beta
    ]
    
    # Problem definition
    prob = cp.Problem(objective, constraints)
    
    # Solve the problem
    prob.solve(solver=cp.GUROBI, TimeLimit=time_limit)
    
    beta_opt = beta.value
    
    return beta_opt


In [11]:
# ALASSO
beta_alasso = alasso(X_train, y_train, kact=10, time_limit=60)
non_negative_count = np.sum(beta_alasso > 0)
non_negative_count

10

In [12]:
non_negative_count

10

## Sorted LASSO- SLOPE
detail is from Sparse index clones via the sorted l1-Norm
SLOPE—ADAPTIVE VARIABLE SELECTION VIA CONVEX OPTIMIZATION

- initial guess beta를 가지고 추정 시작
- 이후 update
- lambda of model: aplha(=sigma of model)(1-i*theta)~ 



In [13]:
# Generate the lambda sequence
def new_lambda(beta_value, alpha, theta, p):
    # Compute the ranks of the absolute values of beta_value
    ranks = np.argsort(np.argsort(-np.abs(beta_value))) + 1
    lambdas = np.array([alpha * norm.ppf(1 - (rank * theta / (2 * p))) for rank in ranks])
    # lambdas = np.array([alpha * norm.ppf(1 - (i * theta / (2 * p))) for i in range(1, p + 1)])
    return lambdas  

In [14]:
def slope(X, y, kact, alpha=1.0, theta=0.1, tol=1e-06, max_iter=10, time_limit=1200, add_loss=0, seed=42):
    """
    Solve the SLOPE optimization problem using Gurobi with an activation constraint.
    
    Parameters:
    X : array-like, shape (T, K)
        The return matrix of benchmark constituents.
    y : array-like, shape (T,)
        The benchmark returns.
    kact : int
        Number of active variables (non-zero coefficients) allowed.
    alpha : float, optional
        Regularization parameter for the lambda sequence.
    theta : float, optional
        Decay rate for the lambda sequence.
    """
    # Number of samples and features
    n, p = X.shape
    weights = np.ones(p)
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        beta_value = beta.X
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            new_weights = np.array(new_lambda(beta_value, alpha, theta, p))
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                break
        else:
            continue
    m.dispose()        
    return beta_value

In [15]:
# SLOPE
beta_slope = slope(X_train, y_train, kact=20, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0xe44256d2
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.32s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [16]:
non_negative_count = np.sum(beta_slope > 0)
non_negative_count

20

# SLOPE-SLC
1. Sorted LASSO의 변형 형태
2. compute for each group the median partial correlation of consituents and keep only groups which are including 75th percent quantile for the equity

In [17]:
from sklearn.linear_model import LassoLarsIC

In [18]:
# def compute_partial_correlation(X, y):
#     # Use LassoLarsIC to find significant features
#     model = LassoLarsIC(criterion='bic')
#     model.fit(X, y)
#     active_features = np.where(model.coef_ != 0)[0]
    
#     partial_corr = np.zeros(X.shape[1])
#     for i in active_features:
#         residual_y = y - model.predict(X[:, active_features])
#         residual_x = X[:, i] - model.predict(X[:, active_features])
#         partial_corr[i] = np.corrcoef(residual_y, residual_x)[0, 1]
    
#     return partial_corr


In [19]:
# def slope_slc(X, y, kact, alpha=1.0, theta=0.1, tol=1e-06, max_iter=10, time_limit=1200, seed=42):
    
#     # Step 1: Solve SLOPE
#     beta_initial = slope(X, y, alpha, theta, tol, max_iter, time_limit, seed)
    
    
#     # Step 2: Compute partial correlation for each asset
#     partial_corr = compute_partial_correlation(X, y)
    
#     # Step 3: Select groups based on median partial correlation
#     median_corr = np.median(partial_corr)
#     threshold = np.percentile(partial_corr, 75)
#     active_indices = np.where(partial_corr >= threshold)[0]
    
#     # Step 4: Rescale SLOPE estimates to sum to 1
#     beta_rescaled = np.zeros_like(beta_initial)
#     beta_rescaled[active_indices] = beta_initial[active_indices]
#     beta_rescaled /= np.sum(beta_rescaled)
    
#     # Step 5: Ensure the number of active weights is less than or equal to kact
#     if np.sum(beta_rescaled > 0) > kact:
#         sorted_indices = np.argsort(beta_rescaled)[::-1]
#         beta_rescaled[sorted_indices[kact:]] = 0
#         beta_rescaled /= np.sum(beta_rescaled)
    
#     return beta_rescaled

In [20]:
# # SLOPE-SLC
# beta_slope_slc = slope_slc(X_train, y_train, kact=20, time_limit=60)

In [21]:
# non_negative_count = np.sum(beta_slope_slc > 0)
# non_negative_count

## MSW_LASSO
detail is from High-dimensional Sprase index tracking based on a multi-step coonvex optimization approach

In [22]:
def msw_lasso(X, y, kact, max_iter=10, tol=1e-12, penalty='MCP', a=2.5, time_limit = 1200, add_loss=0, seed=42):
    n, p = X.shape
    weights = np.ones(p)
    
    def p_lambda(beta_j, lam, a):
        #MCP: b, a =2.5
        #SCAD: a=3.7
        #log-M: epsilon = 1.67*10^-5
        #lq: q=0.1
        if penalty == 'MCP':
            b = a 
            return lam * beta_j - (beta_j**2) / (2 * b) if beta_j <= b * lam else (b * lam**2) / 2
        elif penalty == 'SCAD':
            if beta_j <= lam:
                return lam * beta_j
            elif beta_j <= a * lam:
                return (-beta_j**2 + 2 * a * lam * beta_j - lam**2) / (2 * (a - 1))
            else:
                return (a + 1) * lam**2 / 2
        elif penalty == 'log-M':
            eps = a
            return lam * np.log(1+np.abs(beta_j)/eps) / np.log(1+(1/eps))
        elif penalty == 'lq':
            q = a 
            return lam * (abs(beta_j) **q) 
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        beta_value = beta.X
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            new_weights = np.array([abs(p_lambda(beta_value[j], 1, a)) for j in range(p)])
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                break
        else:
            continue
    m.dispose()         
    return beta_value

In [23]:
# # msw_lasso
beta_msw_lasso = msw_lasso(X_train, y_train, kact=20, penalty='MCP', a=2.5, max_iter=3, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0xe44256d2
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.35s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [24]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

20

In [25]:
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10, penalty='SCAD', a=3.7, max_iter=3, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0x3808f38d
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.21s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [26]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

12

In [27]:
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10, penalty='log-M', a=1.67*(10**-5), max_iter=3, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0x3808f38d
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.20s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [28]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

12

In [29]:
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10, penalty='lq', a=0.1, max_iter=3, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0x3808f38d
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.33s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [30]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

12

# Backtest
## Experiment result 
- Turn over 비용
- Tracking error(for out of sample)
- Computing time(running time second)

In [31]:
def evaluate_portfolio(X, y, beta):
    tracking_error = np.sqrt(np.mean((y - X @ beta)**2))
    turnover = np.sum(np.abs(beta[1:] - beta[:-1])) / len(beta)
    return tracking_error, turnover

In [32]:
def rolling_window_backtest(stock_returns, stock_index, train_window, test_window, kact, max_iter=10, time_limit=60):
    results = {'CARD': [], 'ALASSO': [], 'SLOPE': [], 'MSW-LASSO-SCAD': [],'MSW-LASSO-MCP': [],'MSW-LASSO-LOGM': [],'MSW-LASSO-lq': [], }
    n = len(stock_returns)
    
    for start in range(0, n - train_window - test_window + 1, test_window):
        train_data = stock_returns.iloc[start:start + train_window]
        test_data = stock_returns.iloc[start + train_window:start + train_window + test_window]
    
        # Filter out columns with NaN values in the current window
        valid_assets_train = train_data.dropna(axis=1)
        valid_assets_test = test_data.dropna(axis=1)
        valid_assets = valid_assets_train.columns.intersection(valid_assets_test.columns)
        
        X_train = valid_assets_train[valid_assets].values
        y_train = stock_index.iloc[start:start + train_window].values
        
        X_test = valid_assets_test[valid_assets].values
        y_test = stock_index.iloc[start + train_window:start + train_window + test_window].values
        # Skip if no valid assets are available
        if X_train.shape[1] == 0 or X_test.shape[1] == 0:
            continue
        
        # # CARD
        beta_card = card(X_train, y_train, kact, time_limit=time_limit)
        te_card, to_card = evaluate_portfolio(X_test, y_test, beta_card)
        results['CARD'].append({'date': stock_returns.index[start + train_window], 'te': te_card, 'to': to_card})
        
        # ALASSO
        beta_alasso = alasso(X_train, y_train, kact, time_limit=time_limit)
        te_alasso, to_alasso = evaluate_portfolio(X_test, y_test, beta_alasso)
        results['ALASSO'].append({'date': stock_returns.index[start + train_window], 'te': te_alasso, 'to': to_alasso})
        
        #SLOPE
        beta_slope = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
        te_slope_slc, to_slope_slc = evaluate_portfolio(X_test, y_test, beta_slope)
        results['SLOPE'].append({'date': stock_returns.index[start + train_window], 'te': te_slope_slc, 'to': to_slope_slc})
        
        # MSW-LASSO -SCAD
        beta_msw_lasso_scad = msw_lasso(X_train, y_train, kact=kact, penalty='SCAD', a=3.7, max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_scad, to_msw_lasso_scad = evaluate_portfolio(X_test, y_test, beta_msw_lasso_scad)
        results['MSW-LASSO-SCAD'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_scad, 'to': to_msw_lasso_scad})
        
        # MSW-LASSO -MCP
        beta_msw_lasso_mcp = msw_lasso(X_train, y_train, kact=kact, penalty='MCP', a=2.5, max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_mcp, to_msw_lasso_mcp = evaluate_portfolio(X_test, y_test, beta_msw_lasso_mcp)
        results['MSW-LASSO-MCP'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_mcp, 'to': to_msw_lasso_mcp})

        # MSW-LASSO-log-M
        beta_msw_lasso_logM = msw_lasso(X_train, y_train, kact=kact, penalty='log-M', a=1.67*10**(-5), max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_logM, to_msw_lasso_logM = evaluate_portfolio(X_test, y_test, beta_msw_lasso_logM)
        results['MSW-LASSO-LOGM'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_logM, 'to': to_msw_lasso_logM})

        # MSW-LASSO-lq
        beta_msw_lasso_lq = msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_lq, to_msw_lasso_lq = evaluate_portfolio(X_test, y_test, beta_msw_lasso_lq)
        results['MSW-LASSO-lq'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_lq, 'to': to_msw_lasso_lq})

        print(f"Window ending {stock_returns.index[start + train_window].date()}:")
        print(f"CARD: TE={te_card:.4f}, TO={to_card:.4f}")
        print(f"ALASSO: TE={te_alasso:.4f}, TO={to_alasso:.4f}")
        print(f"SLOPE: TE={te_slope_slc:.4f}, TO={to_slope_slc:.4f}")
        print(f"MSW-LASSO-SCAD: TE={te_msw_lasso_scad:.4f}, TO={to_msw_lasso_scad:.4f}")
        print(f"MSW-LASSO-MCP: TE={te_msw_lasso_mcp:.4f}, TO={to_msw_lasso_mcp:.4f}")
        print(f"MSW-LASSO-logM: TE={te_msw_lasso_logM:.4f}, TO={to_msw_lasso_logM:.4f}")
        print(f"MSW-LASSO-lq: TE={te_msw_lasso_lq:.4f}, TO={to_msw_lasso_lq:.4f}")
    
    return results


In [33]:
def run_experiments(X, y, train_window, test_window, kact_values, max_iter, time_limit, save_path="../results/s&p500"):
    timestamp = datetime.now().strftime('%m%d%H%M%S')
    for kact in kact_values:
        results = rolling_window_backtest(X, y, train_window, test_window, kact, max_iter, time_limit)
        # 각 알고리즘별 결과를 데이터프레임으로 변환
        dataframes = {}
        for key, value in results.items():
            df = pd.DataFrame(value)
            df_name = f"{key}_{kact}_{timestamp}"
            dataframes[df_name] = df
    
        # 데이터프레임을 CSV 파일로 저장
        for df_name, df in dataframes.items():
            file_path = os.path.join(save_path, f"{df_name}.csv")
            df.to_csv(file_path, index=False)
            print(f"Saved {df_name} to {file_path}")
    
    return dataframes

In [ ]:
# kact_values = [10, 20, 30, 40, 50] -> 5h 정도 걸림
kact_values = [10]
train_window = 250
test_window =21
# DATA usage
X = stock_returns.drop(columns=['^GSPC'])
y = stock_returns['^GSPC']


In [ ]:
run_experiments(X, y, train_window, test_window, kact_values, max_iter=10, time_limit=60, file_path="../results/s&p500")

### 시각화

In [ ]:
df_results = pd.read_csv('../results/s&p500_MSW-LASSO-MCP_kact_10.csv')
df_results.head()

In [ ]:
df_results = pd.read_csv('../results_s&p500_kact_20.csv')
# Plot the results
plt.figure(figsize=(14, 7))

for key in df_results:
    plt.plot(df_results[key]['te'], label=f'{key} Tracking Error')

plt.title('Tracking Error Over Time')
plt.xlabel('Date')
plt.ylabel('Tracking Error')
plt.legend()
plt.show()

plt.figure(figsize=(14, 7))

for key in df_results:
    plt.plot(df_results[key]['to'], label=f'{key} Turnover')

plt.title('Turnover Over Time')
plt.xlabel('Date')
plt.ylabel('Turnover')
plt.legend()
plt.show()


# RMIT(Rank Meets Index tracking model)

1. RMIT model makes n samples of B(weight of porfolio) -> using n Indextracking model(with diffent random seed or epoch)
2. for n num of B it test tracking error using train set(most recent 1 month)
3. for n num of B it test tracking error using valid set(1 month)
4. compare rank between 2,3 and add it's loss == rank Loss + origin loss 
5. repeat 1~3 which add rank loss, stop when beta does not change(small then epsilon) 
-> for last choose best beta(smallest beta) 

### Rank LOSS

In [55]:
def pointwise_ranking_loss(y_true, y_pred):
    """
    Pointwise ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    
    Returns:
    float: Pointwise ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = np.mean((y_true - y_pred) ** 2)
    return loss

In [56]:
def pairwise_ranking_loss(y_true, y_pred, margin=1.0):
    """
    Pairwise ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    margin (float): Margin for ranking.
    
    Returns:
    float: Pairwise ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = 0.0
    n = len(y_true)
    for i in range(n):
        for j in range(n):
            if y_true[i] > y_true[j]:
                loss += max(0, margin + y_pred[j] - y_pred[i])
    return loss / (n * (n - 1))

In [57]:
def pairwise_difference_ranking_loss(y_true, y_pred):
    """
    Pairwise difference ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    
    Returns:
    float: Pairwise difference ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = 0.0
    n = len(y_true)
    for i in range(n):
        for j in range(n):
            loss += (y_true[i] - y_true[j] - (y_pred[i] - y_pred[j])) ** 2
    return loss / (n * (n - 1))


In [58]:
def listwise_ranking_loss(y_true, y_pred, tau=1.0):
    """
    Listwise ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    tau (float): Temperature parameter.
    
    Returns:
    float: Listwise ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    def softmax(x, tau=1.0):
        e_x = np.exp(x / tau)
        return e_x / e_x.sum()

    true_prob = softmax(-y_true, tau)
    pred_prob = softmax(-y_pred, tau)
    loss = -np.sum(true_prob * np.log(pred_prob))
    return loss


In [59]:

def rank_loss(rankloss_type, y_true, y_pred,):
    """
    Return the appropriate ranking loss function based on the provided rankloss type.
    
    Parameters:
    rankloss (str): Type of ranking loss ('pointwise', 'pairwise', 'listwise').
    
    Returns:
    rank error rank.
    """
    if rankloss_type == 'pointwise':
        return pointwise_ranking_loss(y_true, y_pred)
    elif rankloss_type == 'pairwise':
        return pairwise_ranking_loss(y_true, y_pred, margin=1.0)
    elif rankloss_type == 'pairwise-diff':
        return pairwise_difference_ranking_loss(y_true, y_pred)
    elif rankloss_type == 'listwise':
        return listwise_ranking_loss(y_true, y_pred, tau=1.0)
    else:
        raise ValueError("Unsupported rankloss type. Choose from 'pointwise', 'pairwise', 'listwise'.")

### sampling Beta

In [60]:
def slope_sampling(X, y, kact, alpha=1.0, theta=0.1, tol=1e-06, max_iter=10, time_limit=1200, add_loss=0, seed=42):
    n, p = X.shape
    beta_sample = []
    weights = np.ones(p)
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        beta_value = beta.X
        beta_sample.append(beta_value)
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            new_weights = np.array(new_lambda(beta_value, alpha, theta, p))
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                break
        else:
            continue
    m.dispose()        
    return beta_sample


def msw_lasso_sampling(X, y, kact, max_iter=10, tol=1e-12, penalty='MCP', a=2.5, time_limit = 1200, add_loss=0, seed=42):
    n, p = X.shape
    weights = np.ones(p)
    beta_sample = []
    def p_lambda(beta_j, lam, a):
        #MCP: b, a =2.5
        #SCAD: a=3.7
        #log-M: epsilon = 1.67*10^-5
        #lq: q=0.1
        if penalty == 'MCP':
            b = a 
            return lam * beta_j - (beta_j**2) / (2 * b) if beta_j <= b * lam else (b * lam**2) / 2
        elif penalty == 'SCAD':
            if beta_j <= lam:
                return lam * beta_j
            elif beta_j <= a * lam:
                return (-beta_j**2 + 2 * a * lam * beta_j - lam**2) / (2 * (a - 1))
            else:
                return (a + 1) * lam**2 / 2
        elif penalty == 'log-M':
            eps = a
            return lam * np.log(1+np.abs(beta_j)/eps) / np.log(1+(1/eps))
        elif penalty == 'lq':
            q = a 
            return lam * (abs(beta_j) **q) 
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        beta_value = beta.X
        beta_sample.append(beta_value)
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            new_weights = np.array([abs(p_lambda(beta_value[j], 1, a)) for j in range(p)])
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                break
        else:
            continue
    m.dispose()         
    return beta_sample

In [61]:
def sampling_Beta(X, y, kact, algorithm_name, max_iter, time_limit, add_loss=0):
# msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit)
#beta_slope = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
# stock_returns, stock_index, train_window, test_window, kact, max_iter=10, time_limit=1200
    if algorithm_name == 'SLOPE':       
        #SLOPE
        beta_sample = slope_sampling(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-SCAD':   
        # MSW-LASSO -SCAD
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='SCAD', a=3.7, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-MCP':      
        # MSW-LASSO -MCP
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='MCP', a=2.5, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-logM':  
        # MSW-LASSO-log-M
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='log-M', a=1.67*10**(-5), max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-lq':  
        # MSW-LASSO-lq
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    
    return beta_sample

In [62]:
def calculate_Beta(X, y, kact, algorithm_name, max_iter, time_limit, add_loss=0):
# msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit)
#beta_slope = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
# stock_returns, stock_index, train_window, test_window, kact, max_iter=10, time_limit=1200
    if algorithm_name == 'SLOPE':       
        #SLOPE
        beta_sample = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
    elif algorithm_name == 'MSW-LASSO-SCAD':   
        # MSW-LASSO -SCAD
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='SCAD', a=3.7, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-MCP':      
        # MSW-LASSO -MCP
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='MCP', a=2.5, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-logM':  
        # MSW-LASSO-log-M
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='log-M', a=1.67*10**(-5), max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-lq':  
        # MSW-LASSO-lq
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    
    return beta_sample

In [63]:
def RMIT(X, y, test_window_size=21, kact=10, algorithm='SLOPE', rank_type='pointwise', n_models=10, max_iter=10, time_limit=60):
    beta = None
    train_errors = []
    valid_errors = []
    rank_loss_value = 0 
    X_train = X[:-2*test_window_size]
    y_train = y[:-2*test_window_size]
    X_train_last = X[-2*test_window_size:-test_window_size]
    y_train_last = y[-2*test_window_size:-test_window_size]
    X_val = X[-test_window_size:]
    y_val = y[-test_window_size:]
    
    for _ in range(max_iter-1): 
        # 1. Generate n samples of B using n regression models & use updated rank_loss 
        B_samples = sampling_Beta(X_train, y_train, kact, algorithm_name=algorithm, max_iter=n_models, time_limit=time_limit, add_loss = rank_loss_value)
        # 2. Test tracking error using train set(1month)
        for B in B_samples:
            te, to = evaluate_portfolio(X_train_last, y_train_last, B)
            train_errors.append(te)
        # 3. Test tracking error using valid set
        valid_errors = []
        for B in B_samples:
            te, to = evaluate_portfolio(X_val, y_val, B)
            valid_errors.append(te)
        # 4. Compare rank between train and valid errors and add its loss
        rank_loss_value = rank_loss(rank_type, train_errors, valid_errors)

    #5. compute last beta using rankloss + origin loss of algorithm 
    # use all data + val
    beta = calculate_Beta(X, y, kact, algorithm_name=algorithm, max_iter=max_iter, time_limit=time_limit, add_loss = rank_loss_value)

    return beta


In [64]:
stock_stocks = stock_returns.drop(columns=['^GSPC'])
stock_index = stock_returns['^GSPC']

train_data = stock_stocks.iloc[:250]

# Filter out columns with NaN values in the current window
valid_assets_train = train_data.dropna(axis=1)
valid_assets = valid_assets_train.columns.intersection(valid_assets_train.columns)

X_train = valid_assets_train[valid_assets].values
y_train = stock_index.iloc[:250].values

In [65]:
## RMIT
beta_RMIT = RMIT(X_train, y_train, test_window_size=21, kact=10,  algorithm='SLOPE', rank_type='pointwise', n_models=10, max_iter=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0x3808f38d
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.29s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv